In [3]:
import pandas as pd

n_campi = 'CAFI'

levantamento = pd.read_excel('consolidacao/levantamento.xlsx')
suap = pd.read_excel('consolidacao/relatorio_CAFI.xlsx')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]
# lista de itens vencidos
vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]


# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código', 'Retificação'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})
# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
display(suap_cross)
display(levantamento_cross)

consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

,codigo,descricao_suap,qtd_suap,ed
0,30.07.862,30.07.351 ÁGUA MINERAL EM GARRAFÃO DE 20 LITRO...,5,07 - GENEROS DE ALIMENTACAO
1,30.16.1000,"ENVELOPE TIPO CARTA CORREIO, 114x229MM, SEM TI...",3990,16 - MATERIAL DE EXPEDIENTE
2,30.16.1002,"ENVELOPE PORTA DOCUMENTO EM PVC, 100 X 75MM, C...",730,16 - MATERIAL DE EXPEDIENTE
3,30.16.1007,"PASTA CANALETA, TAMANHO A4, TRANSPARENTE, COR ...",454,16 - MATERIAL DE EXPEDIENTE
4,30.16.1009,"CAIXA CORRESPONDÊNCIA, TIPO TRIPLA, MATERIAL A...",44,16 - MATERIAL DE EXPEDIENTE
...,...,...,...,...
310,30.42.46,TRENA FIBRA DE VIDRO (30METROS),1,42 - FERRAMENTAS
311,30.42.891,"FERRAMENTA DE INSERÇÃO (PUSH-DOWN), TIPO USO E...",5,42 - FERRAMENTAS
312,30.42.977,ALICATE MULTIFUNCIONAL PARA DESENCAPAR FIOS E ...,1,42 - FERRAMENTAS
313,30.44.1,"FITA FAIXA ZEBRADA, POLIETILENO, COM LISTRAS A...",6,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS


,codigo,descricao_lev,qtd_lev,servidor
0,NaN,NaN,NaN,NaN
1,30.17.195,TONER HP COMPATÍVEL Q6511X COR PRETA,3.0,djalma.gonzaga@ead.ifpe.edu.br
2,30.17.544,"FITA PARA IMPRESSORA (RIBBON), MATERIAL POLITE...",20.0,jose.paulo@reitoria.ifpe.edu.br
3,30.17.318,"TONER SAMSUNG ORIGINAL SCX-4521D3, SCX 4521F C...",2.0,djalma.gonzaga@ead.ifpe.edu.br
4,30.17.194,TONER HP COMPATÍVEL Q2963A/Q3963A COR MAGENTA ...,5.0,djalma.gonzaga@ead.ifpe.edu.br
...,...,...,...,...
303,30.16.1337,"PAPEL SULFITE, MATERIAL CELULOSE VEGETAL, GRAM...",201.0,paulo.uchoa@reitoria.ifpe.edu.br
304,30.16.1694,"PAPEL A4, MATERIAL PAPEL ALCALINO, COMPRIMENTO...",86.0,paulo.uchoa@reitoria.ifpe.edu.br
305,30.16.2718,"PAPEL A4, TIPO COLORIDO, CORES VARIADAS, 297X2...",28.0,paulo.uchoa@reitoria.ifpe.edu.br
306,30.16.2402,PAPEL COLORSET. PCT 20FL,10.0,paulo.uchoa@reitoria.ifpe.edu.br


In [5]:
consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo', 'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.14.117,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",16,14 - MATERIAL EDUCATIVO E ESPORTIVO,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",14,elaine.leite@afogados.ifpe.edu.br,both,2
1,30.14.120,BOLA TENIS DE MESA,17,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOLA TENIS DE MESA,12,elaine.leite@afogados.ifpe.edu.br,both,5
2,30.14.121,REDE ESPORTE BASQUETE,8,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE BASQUETE,7,elaine.leite@afogados.ifpe.edu.br,both,1
3,30.14.122,REDE ESPORTE FUTSAL,10,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE FUTSAL,10,elaine.leite@afogados.ifpe.edu.br,both,0
4,30.14.123,REDE ESPORTE FUTEBOL DE CAMPO,3,14 - MATERIAL EDUCATIVO E ESPORTIVO,REDE ESPORTE FUTEBOL DE CAMPO,3,elaine.leite@afogados.ifpe.edu.br,both,0
...,...,...,...,...,...,...,...,...,...
631,30.42.972,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,39,42 - FERRAMENTAS,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,39,dario.frazao@afogados.ifpe.edu.br,both,0
632,30.42.973,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,40,42 - FERRAMENTAS,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,40,dario.frazao@afogados.ifpe.edu.br,both,0
633,30.44.67,"Fita adesiva, material propileno, 20mm x 30m, ...",11,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,"Fita adesiva, material propileno, 20mm x 30m, ...",10,dario.frazao@afogados.ifpe.edu.br,both,1
634,30.44.69,PLACA DE SINALIZAÇÃO DE AMBIENTE,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,PLACA DE SINALIZAÇÃO DE AMBIENTE,2,dario.frazao@afogados.ifpe.edu.br,both,0


,codigo,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.14.117,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",16,14,2,pedido no suap
1,30.14.120,BOLA TENIS DE MESA,17,12,5,pedido no suap
2,30.14.121,REDE ESPORTE BASQUETE,8,7,1,pedido no suap
3,30.14.122,REDE ESPORTE FUTSAL,10,10,0,adequado
4,30.14.123,REDE ESPORTE FUTEBOL DE CAMPO,3,3,0,adequado
...,...,...,...,...,...,...
631,30.42.972,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,39,39,0,adequado
632,30.42.973,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,40,40,0,adequado
633,30.44.67,"Fita adesiva, material propileno, 20mm x 30m, ...",11,10,1,pedido no suap
634,30.44.69,PLACA DE SINALIZAÇÃO DE AMBIENTE,2,2,0,adequado
